(c) Julia Riedl 2025
The code iterates through folders of recording dates and contained single movie folders.

It extracts well position by binarising the image, with a threshold which finds round object of specific size.
Well position is corrected for deviation from theoretical ones. 

For each well pixels in areas of user-defined sizes are subtracted from "inc" frames later to calculate pixel variance.
This is done in "framestep" intervals, e.g.  every 2 seconds if framestep=60 for  30hz movie data.

Data are stored in dictionary "MovieVarData"


In [1]:
#import libraries
import os
import numpy as np
import cv2
from skimage import filters, measure
from scipy.ndimage import gaussian_filter
import matplotlib.pyplot as plt
from skimage.draw import disk

In [2]:
# Parameters
imout = 1 #figure output 1=yes, 0=no
tl = 0.8  # Binarisation threshold level factor starting level
well_r = 120  # Well radius in pixels
inc = 2  # Frame increment between frames to be subtracted
framestep=3 # frame interval between frame samples for variance 
sigma = 0.6  # Gaussian filter strength, the higher the stronhger the filter


In [4]:
# (1)indicate data location
# initialise dictionary holding data for various movies in given folder
parentF = r'/Volumes/behavgenom$/John/data_exp_info/NHR/Bonnie_compounds/data/video/250213_40h/not_viscous_1/single_wells'
os.chdir(parentF)
dayfolders= os.listdir(parentF)
dayfolders.sort()

#try to load partial data back if existing:
#load data back

try:
    import cPickle as pickle
except ImportError:  # Python 3.x
    import pickle

try:
    with open('MovieVarDataP.p', 'rb') as fp:
        MovieVarData = pickle.load(fp)
        print('loaded previous data')
        days = list(MovieVarData.keys())
        currd=MovieVarData.get(days[-1])
        print("last recording day analysed:" + str(days[-1]))
        dm=list(currd.keys())
        print(dm[-1])
        #find last recording day analysed
        di = [i for i,s in enumerate(dayfolders)if days[-1] in s]
        di=di[0]
        print(di)
        # di=int(di)
        #find last movie analysed
        mov_idx=len(currd)
        # mi=int(mi)

except Exception:
    print('No MovieVarData found in current direcory.')
    MovieVarData = {}
    di=1
    mov_idx=1
    print(di)


No MovieVarData found in current direcory.
1


In [116]:
# BWLevel = filters.threshold_otsu(frame)*0.9
# fn=frame[::10, ::10, 0]
# BW = fn > BWLevel
os.chdir(os.path.join(parentF,DD, movie))
print(os.getcwd())
#MovieVarData[movie]=plate
np.save('MovieVarData.npy', MovieVarData)
os.chdir("..")

ps=fn.shape
print(wellfolders)
rr, cc = disk((round(ps[0]/2) , round(ps[1]/2)), round(ps[0]/2.5), shape=ps)
mask = np.zeros(pic_o.shape, dtype=bool)
mask[rr,cc] = 1
plt.imshow(mask)
plt.colorbar()

plt.figure
#plt.imshow(difpic)
plt.title(ti)



/Volumes/behavgenom$/John/data_exp_info/NHR/Bonnie_compounds/data/video/250212_20h/not_viscous_1


OSError: [Errno 28] No space left on device: 'MovieVarData.npy'

In [6]:
#---main code:----
# (1) goes through folders of recording days and  loads movies 
# (2) finds X and Y offset from theoretical well centroids as given in paranmeter section
# (3) creates a mask for each well with corrected well centroid of given radius from paramter section 
# (4) subtracts pixel values from each well from next movie frame with interval defined in parameters
# ----------

# (1) change into folders containing movies
os.chdir(parentF)
dayfolders= os.listdir(parentF)
dayfolders.sort()
dayfolders=np.array(dayfolders[di:])
print('found subdirectories:'  + str(len(dayfolders)))
dc=1

for DD in dayfolders: #go through recording dates
    
    os.chdir(os.path.join(parentF, DD))
    print('current day dir: ' + DD)
    subdirectory=os.getcwd()
    moviefolders= os.listdir(subdirectory)
    moviefolders.sort()

    if dc>1:
        mov_idx=1
    print('starting at folder#' + str(mov_idx) + ' of ' + str(len(moviefolders)) )
    moviefolders=moviefolders[mov_idx:]

    dc=dc+1

    # go through recording day folders as given in days variable
    mc=0   
    plate={}
    for movie in moviefolders:
        mc=mc+1
        os.chdir(os.path.join(parentF,DD, movie))
        subdirectory=os.getcwd()
        wellfolders=os.listdir(subdirectory)
        wellfolders.sort()
        wellfolders=wellfolders[1:]

        for wf in wellfolders:
            if os.path.isdir(os.path.join(parentF,DD, movie, wf)):
                os.chdir(os.path.join(parentF,DD, movie, wf))

            # Find and load the .mp4 file
            movie_files = [f for f in os.listdir() if f.endswith('.avi')]
            if not movie_files:
                contimovies =dayfolders[gmi]

            v = cv2.VideoCapture(movie_files[0])
            print("Loaded movie#",mc, ';',wf)

            framenums = 40
            well_diffs=np.zeros(round(framenums/framestep)+1)
            FC = 1

            for fr in range(1, framenums, framestep): # pixel variance every "framestep" frames(e.g.60=2sec)
                #  at frame 1: asess well position offset from given theoretical well centroids
                # by binarising image with correct threshold to find round structures of expected size
                if FC == 1:     
                    # Set the video position to the desired frame
                    v.set(cv2.CAP_PROP_POS_FRAMES, fr)
                    # Read the frame
                    ret, frame = v.read()
                    if not ret:
                        print('couldnt read frame from video')
                        break

                #make mask
                fn=frame[::2, ::2, 0]
                ps=fn.shape
                rr, cc = disk((round(ps[0]/2) , round(ps[1]/2)), round(ps[0]/2.5), shape=ps)
                mask = np.zeros(pic_o.shape, dtype=bool)
                mask[rr,cc] = 1
                                # for ti in np.arange(tl, tl * 1.1, 0.04):
                #     BWLevel = filters.threshold_otsu(picin) * ti
                #     BW = picin > BWLevel
                #     labeled, num = measure.label(BW, return_num=True)
                #     wellstats = measure.regionprops(labeled)

                #     min_well_area = picin.size * 0.3
                #     max_well_area = picin.size * 0.6

                #     WellIdx = [i for i, region in enumerate(wellstats)
                #                 if min_well_area < region.area < max_well_area and region.eccentricity < 0.5]

                #     if len(WellIdx) == 1:
                #         print(f"{len(WellIdx)} wells found")
                #         mask = np.zeros(pic_o.shape, dtype=bool)
                #         mask[labeled[WellIdx]] = 1   
                #         break
                        
                        # plt.imshow(labeled)
                        # plt.title(f"No wells found, threshold = {BWLevel}")
                        # plt.show()
                        # plt.close()
                    # else:
                    #     print(f"No wells found, threshold = {BWLevel}")
                # if ti==tl * 1.1 and len(WellIdx) < 1:
                #     ps=(picin.shape)
                #     print("No wells detected, took default centroids")
                #     plt.imshow(labeled)
                #     plt.scatter(x=round(ps[1]/2),y=round(ps[0]/2),c='g',s=20)
                
            

                # Calculate pixel variance over time for each well
                diff_pix = [None] * 3
                nv = np.full((len(binedges) - 1, 16), np.nan)

                # Set the video position to the desired frame
                v.set(cv2.CAP_PROP_POS_FRAMES, fr)
                # Read the frame
                ret, frame_o = v.read()
                if not ret:
                    print('couldnt read frame from video')
                    break

                pic_o = frame_o[:, :, 0].astype(float)
                pic_of = gaussian_filter(pic_o, sigma)

                # Set the video position to the desired frame
                v.set(cv2.CAP_PROP_POS_FRAMES, fr+inc)
                # Read the frame
                ret, frame_1 = v.read()
                if not ret:
                    print('couldnt read frame from video')
                    break
                
                pic1 = frame_1[:, :, 0].astype(float)
                pic1f = gaussian_filter(pic1, sigma)

                pic1 = pic1f[::2, ::2]
                pic_o = pic_of[::2, ::2]
                difpic = np.abs(pic1 - pic_o)
        
                mi = np.nonzero(mask)
                diff_pix=difpic[mi]
                dpn=len(diff_pix[diff_pix > 4]) #number of pixels with diff >4
                difp_norm=dpn/sum(sum(mask)) #normalised to number of pixels in well
                # change to simply pixel diffs>4
 
                well_diffs[FC]=difp_norm
                FC += 1
          

                if FC % 4 == 0:
                    print(f"Frames analysed: {FC}")
            #save
            plate[wf]=well_diffs
        #MovieVarData.setdefault(DD, {}).setdefault(movie, {})[currcam] = well_diffs[1:]
        #ols:MovieVarData.setdefault(DD, movie , {})[currcam] = well_diffs[1:] #save

        MovieVarData[movie]=plate
        os.chdir("..")
    #save:
    # currset = {movie[-7:]}
    # currmovie = f"{movie[0:-9]}"
    np.save('MovieVarData.npy', MovieVarData)
    os.chdir("..")
# final Save MovieVarData
np.save('MovieVarData.npy', MovieVarData)

found subdirectories:72
current day dir: A1
starting at folder#1 of 1
current day dir: A10
starting at folder#1 of 1
current day dir: A11
starting at folder#1 of 1
current day dir: A12
starting at folder#1 of 1
current day dir: A2
starting at folder#1 of 1
current day dir: A3
starting at folder#1 of 1
current day dir: A4
starting at folder#1 of 1
current day dir: A5
starting at folder#1 of 1
current day dir: A6
starting at folder#1 of 1
current day dir: A7
starting at folder#1 of 1
current day dir: A8
starting at folder#1 of 1
current day dir: A9
starting at folder#1 of 1
current day dir: B1
starting at folder#1 of 1
current day dir: B10
starting at folder#1 of 1
current day dir: B11
starting at folder#1 of 1
current day dir: B12
starting at folder#1 of 1
current day dir: B2
starting at folder#1 of 1
current day dir: B3
starting at folder#1 of 1
current day dir: B4
starting at folder#1 of 1
current day dir: B5
starting at folder#1 of 1
current day dir: B6
starting at folder#1 of 1
curr

In [7]:
os.chdir(parentF)
currdir=os.getcwd()
print(currdir)
print(DD)

/Volumes/behavgenom$/John/data_exp_info/NHR/Bonnie_compounds/data/video/250213_40h/not_viscous_1/single_wells
F9


In [118]:
#save:
os.chdir('/Users/jriedl/Desktop/NHR_temp')
currdir=os.getcwd()
print(currdir)

#'MovieVarData.npy', MovieVarData

try:
    import cPickle as pickle
except ImportError:  # Python 3.x
    import pickle

with open('MovieVarDataP.p', 'wb') as fp:
    pickle.dump(MovieVarData, fp, protocol=pickle.HIGHEST_PROTOCOL)


/Users/jriedl/Desktop/NHR_temp
